# <center>  Pose Estimation with MediaPipe </center>
<hr>

### 0. Import Dependencies

In [1]:
import mediapipe as mp
import cv2
import numpy as np
import math

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

### 1. Joint Detection 

<img src="https://google.github.io/mediapipe/images/mobile/pose_tracking_full_body_landmarks.png" >

In [42]:
cap = cv2.VideoCapture(0)
## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

### 1. Angle calculation 

##### 1.a )  AngLe with 3 Dimension

In [49]:
def calculate_angle(F,M,E):
    First = np.array(F) 
    Midle = np.array(M) 
    End = np.array(E) 
    
    v1 = [First[0] - Midle[0], First[1] - Midle[1],First[2] - Midle[2]]

    v2 = [End[0] - Midle[0], End[1] - Midle[1],End[2] - Midle[2]]

    lenght1  = math.sqrt(v1[0] * v1[0] + v1[1] * v1[1] + v1[2] * v1[2])

    lenght2  = math.sqrt(v2[0] * v2[0] + v2[1] * v2[1] + v2[2] * v2[2])
    
    radian = math.acos(np.dot(v1,v2)/(lenght1 * lenght2)) # Radian
    angle = radian * (180/np.pi) #degree
    
    return round (angle) 

In [50]:
def calculate_angle2(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [51]:
Elbow_Left_angle = calculate_angle(shoulder_Left, elbow_Left, wrist_Left)
Elbow_Left_angle

105

##### 1.b )  AngLe with 2 Dimension

### 2. Real time Pose Estimation 

#### 2.A The angles in real time

#### 2.B The angles in real time (+ or -)

In [52]:
tuple(np.multiply(elbow, [640, 480]).astype(int))


(776, 408)

In [53]:
shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].z]
elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].z]
wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].z]

In [54]:
shoulder

[0.9030404090881348, 0.26676493883132935, -1.3897978067398071]

In [55]:
calculate_angle(shoulder, elbow, wrist)


126

In [56]:
landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]


x: 1.213816523551941
y: 0.8516172170639038
z: -1.2644582986831665
visibility: 0.5706676244735718

In [57]:
cap = cv2.VideoCapture(0)
## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            
            # Calculate angle
            angle = calculate_angle2(shoulder, elbow, wrist)
            
            # Visualize angle
            cv2.putText(image, str(angle), 
                           tuple(np.multiply(elbow, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
                       
        except:
            pass
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

### 2.C Walks in real time 

* **If both legs are visible;**
* **If one of the legs is placed in front of the other;**
* **If the distance between the right ankle and the right hip is less than the distance between the left ankle and the left hip.** (Or the Distance between the Ankle Left and Hip Left <  Distance between the Ankle Right  and Hip right) ;
* **One knee joint is in the neutral position and the other is in the flexion phase.** (one leg 180° the other is less)


#### 2.C.1 Try fisrt Condition with Legs angles

In [9]:
# Count list of Steps 
def stepsCount (walksList):
    counter = 0 
    for i in range (len(walksList)):
        if ((walksList[i] == 'True') and (i == len(walksList)-1)):
            print (walksList[i])
            counter += 1 
            break ;          
        if ((walksList [i] == 'True' )  and (walksList [i+1] == 'False' )):
            print(walksList[i])
            counter = counter +1 
    return counter 

In [10]:
cap = cv2.VideoCapture(0) #To use webcam

# Steps counter variables
counter = 0 
steps = []

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
        #print(results)
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates Right Knee
            hip_Right =  [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            knee_Right = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            ankle_Right =[landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
            
             # Get coordinates Left Knee
            hip_Left =  [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            knee_Left = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            ankle_Left =[landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            
            
            #Angle Calculation

            # Calculate Right knee
            Knee_Right_angle = calculate_angle(hip_Right, knee_Right, ankle_Right)
        
            #Calculate Left knee
            Knee_Left_angle = calculate_angle(hip_Left, knee_Left, ankle_Left)

    
            # Steps counter 
            if ((Knee_Right_angle >= 170) and  (Knee_Left_angle<170)) :
                steps.append(True)
                counter +=1 
            elif ((Knee_Left_angle >= 170) and  (Knee_Right_angle<170)):
                steps.append(True)
                counter =+1 
            else :
                steps.append(False)
         
            # Visualize Angles ******************************************************
            # Visualize  180°-Knee left angle
            cv2.putText(image, str(180-Knee_Left_angle), 
                        tuple (np.multiply(knee_Left, [640, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (245,117,66), 2, cv2.LINE_AA)
            
            
            # Visualize  180°-Knee Right angle
            cv2.putText(image, str(180-Knee_Right_angle), 
                        tuple (np.multiply(knee_Right, [640, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (245,117,66), 2, cv2.LINE_AA)               
        except:
            pass
   
        # Steps visualisation
        cv2.putText(image, 'Steps', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        
        cv2.putText(image, str(stepsCount(steps)), 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(255,255,255), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(0,0,255), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Pose Estimation', image)

        if cv2.waitKey(10) & 0xFF == ord('k'):
            break

    cap.release()
    cv2.destroyAllWindows()

#### 2.B.2 Try Second Condition with Legs y axes

In [72]:
cap = cv2.VideoCapture(0) #To use webcam

# Steps counter variables
counter = 0 
steps = []
boolStep = ''
## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
        #print(results)
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates Right Knee
            hip_Right =  [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].z]
            knee_Right = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].z]
            ankle_Right =[landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].z]
            
             # Get coordinates Left Knee
            hip_Left =  [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].z]
            knee_Left = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].z]
            ankle_Left =[landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].z]
            
            
            # Angle Calculation
            # Calculate Right knee
            Knee_Right_angle = calculate_angle(hip_Right, knee_Right, ankle_Right)
        
            #Calculate Left knee
            Knee_Left_angle = calculate_angle(hip_Left, knee_Left, ankle_Left)

    
            # Steps counter 
            if ((Knee_Right_angle >= 170) and  (Knee_Left_angle<170) and (ankle_Right[1] > ankle_Left[1]) ) :
                steps.append(True)
                boolStep = 'True'
                counter +=1 
            elif ((Knee_Left_angle >= 170) and  (Knee_Right_angle<170) and  (ankle_Right[1] < ankle_Left[1])):
                steps.append(True)
                boolStep = 'True'
                counter =+1 
            else :
                steps.append(False)
                boolStep = 'True'

         
            # Visualize Angles ******************************************************
            # Visualize  180°-Knee left angle
            cv2.putText(image, str(180-Knee_Left_angle), 
                        tuple (np.multiply(knee_Left, [640, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (245,117,66), 2, cv2.LINE_AA)
            
            
            # Visualize  180°-Knee Right angle
            cv2.putText(image, str(180-Knee_Right_angle), 
                        tuple (np.multiply(knee_Right, [640, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (245,117,66), 2, cv2.LINE_AA)               
        except:
            pass
   
        # Steps visualisation
        cv2.putText(image, 'Steps', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        
        cv2.putText(image, str(boolStep), 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(255,255,255), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(0,0,255), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Pose Estimation', image)

        if cv2.waitKey(10) & 0xFF == ord('k'):
            break

    cap.release()
    cv2.destroyAllWindows()